In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [3]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [4]:
val_df['Image_ID'].nunique()

56

In [5]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("zindi_challenge_cacao/train*/weights/best.pt"))[-1]
BEST_PATH

'zindi_challenge_cacao/train/weights/best.pt'

In [6]:
# Validate the model on the validation set
BEST_CFG = sorted(glob("zindi_challenge_cacao/train*/args.yaml"))[-1]
BEST_CFG

'zindi_challenge_cacao/train/args.yaml'

In [7]:
# Load the trained YOLO model
model = YOLO(BEST_PATH)

# model = model.eval()

In [8]:
for flag, v in ExifTags.TAGS.items():
	if v == "Orientation":
		break


def load_image_(filepath):
	image = Image.open(filepath)
	# return image

	exif = image._getexif()
	if exif is None:
		return image

	orientation_value = exif.get(flag, None)

	if orientation_value == 3:
		image = image.rotate(180, expand=True)
	elif orientation_value == 6:
		image = image.rotate(270, expand=True)
	elif orientation_value == 8:
		image = image.rotate(90, expand=True)
	return image

from ultralytics.utils.patches import imread
import cv2

def load_image(filepath):
	# return load_image_(filepath)
	return imread(filepath, cv2.IMREAD_COLOR)


flag

274

In [9]:
import yaml
with open(BEST_CFG, 'r') as f:
	cfg: dict = yaml.safe_load(f)
	print(cfg)

{'task': 'detect', 'mode': 'train', 'model': 'yolo11l.pt', 'data': 'data.yaml', 'epochs': 200, 'time': None, 'patience': 20, 'batch': 16, 'imgsz': 1024, 'save': True, 'save_period': -1, 'cache': False, 'device': [0, 1], 'workers': 8, 'project': 'zindi_challenge_cacao', 'name': 'train', 'exist_ok': False, 'pretrained': True, 'optimizer': 'auto', 'verbose': True, 'seed': 0, 'deterministic': True, 'single_cls': False, 'rect': False, 'cos_lr': False, 'close_mosaic': 10, 'resume': False, 'amp': True, 'fraction': 1.0, 'profile': False, 'freeze': None, 'multi_scale': True, 'overlap_mask': True, 'mask_ratio': 4, 'dropout': 0.3, 'val': True, 'split': 'val', 'save_json': False, 'conf': None, 'iou': 0.42361207118859606, 'max_det': 150, 'half': True, 'dnn': False, 'plots': True, 'source': None, 'vid_stride': 1, 'stream_buffer': False, 'visualize': False, 'augment': True, 'agnostic_nms': False, 'classes': None, 'retina_masks': False, 'embed': None, 'show': False, 'save_frames': False, 'save_txt': F

In [10]:
# Batch size for predictions
batch_size = 16

cfg["device"] = "cuda"
cfg["batch"] = batch_size
cfg["conf"] = 0.
cfg["verbose"] = False

cfg.pop("source")
# cfg.pop("batch_size")
cfg.pop("visualize")

keys = list(cfg.keys())
for col in keys:
	if "show" in col or "save" in col:
		cfg.pop(col)

In [11]:
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = [i for i in os.listdir(test_dir_path) if not i.endswith(".npy")]

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []


with torch.no_grad():
	# Process images in batches
	for i in tqdm(range(0, len(image_files), batch_size)):
		batch_files = image_files[i:i + batch_size]
		batch_images = [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

		# Make predictions on the batch of images
		results = model.predict(
			batch_images,
			# seed=cfg['seed'],
			# deterministic=cfg['deterministic'],
			# close_mosaic=cfg['close_mosaic'],
			# mask_ratio=cfg['mask_ratio'],
			# iou=cfg['iou'],
			# max_det=cfg['max_det'],
			# format=cfg['format'],
			# nms=cfg['nms'],
			# half=cfg['half'],
			# agnostic_nms=cfg['agnostic_nms'],
			# # rect=True,
			# # mode="predict",
			# amp=cfg['amp'],
			# cfg=BEST_CFG,
			**cfg,

			# conf=0.,
			# # imgsz=cfg['imgsz'],
			# batch=batch_size,
			# verbose=False,
			# # device=cfg['device'],
			# device="cuda",
		)

		# Iterate through each result in the batch
		for img_file, result in zip(batch_files, results):
			boxes = result.boxes.xyxy.tolist() if result.boxes else []  # Bounding boxes in xyxy format
			classes = result.boxes.cls.tolist() if result.boxes else []  # Class indices
			confidences = result.boxes.conf.tolist() if result.boxes else []  # Confidence scores
			names = result.names  # Class names dictionary

			if boxes:  # If detections are found
				for box, cls, conf in zip(boxes, classes, confidences):
					x1, y1, x2, y2 = box
					detected_class = names[int(cls)]  # Get the class name from the names dictionary

					# Add the result to the all_data list
					all_data.append(
						{
							"Image_ID": str(img_file),
							"class": detected_class,
							"confidence": conf,
							"ymin": y1,
							"xmin": x1,
							"ymax": y2,
							"xmax": x2,
						}
					)
			else:  # If no objects are detected
				all_data.append(
					{
						"Image_ID": str(img_file),
						"class": "None",
						"confidence": None,
						"ymin": None,
						"xmin": None,
						"ymax": None,
						"xmax": None,
					}
				)


100%|██████████| 4/4 [00:07<00:00,  1.87s/it]


In [12]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [13]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_MYSxE2.jpg,healthy,0.872559,8.765625,371.34375,3264.000000,1933.218750
1,ID_MYSxE2.jpg,healthy,0.088806,0.996094,1761.09375,458.203125,2448.000000
2,ID_MYSxE2.jpg,healthy,0.019089,2862.375000,283.68750,3264.000000,1216.031250
3,ID_MYSxE2.jpg,healthy,0.018341,0.000000,0.00000,207.585938,466.171875
4,ID_MYSxE2.jpg,cssvd,0.015541,2285.437500,1877.43750,2907.000000,2448.000000


In [14]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,8400.000000,8400.000000,8400.000000,8400.000000,8400.000000
mean,0.009509,829.951959,665.081140,1203.816688,1037.340690
std,0.077093,958.262124,803.931225,1094.589469,941.553917
min,0.000002,0.000000,0.000000,6.842773,0.000000
25%,0.000009,21.423827,6.562500,345.993164,313.813454
50%,0.000026,502.656250,389.375000,887.859375,810.000000
75%,0.000121,1221.831940,958.000000,1716.872955,1476.222656
max,0.923828,4055.437500,3968.999756,4128.000000,4032.000000


In [15]:
sub['class'].value_counts()

class
healthy        3343
cssvd          2854
anthracnose    2203
Name: count, dtype: int64

In [16]:
def load_yolo_labels(label_folder):
	label_data = {}
	label_folder = Path(label_folder)
	paths = [i for i in label_folder.glob("*") if i.suffix != ".npy"]

	for label_file in paths:
		with open(label_file, "r") as file:
			annotations = []
			for line in file:
				parts = line.strip().split()
				if len(parts) == 5:
					class_id, x_center, y_center, width, height = map(float, parts)
					annotations.append({
						"class_id": int(class_id),
						"x_center": x_center,
						"y_center": y_center,
						"width": width,
						"height": height
					})
				else:
					print(f"Skipping line in {label_file}: {line.strip()}")
			label_data[label_file.stem] = annotations
	# Convert the label data to a pandas DataFrame
	label_df = []
	for image_id, annotations in label_data.items():
		for annotation in annotations:
			label_df.append({
				"Image_ID": image_id,
				"class_id": annotation["class_id"],
				"x_center": annotation["x_center"],
				"y_center": annotation["y_center"],
				"width": annotation["width"],
				"height": annotation["height"]
			})

	label_df = pd.DataFrame(label_df)
	return label_df

# Example usage
label_folder = VAL_LABELS_DIR
labels = load_yolo_labels(label_folder)
labels.sample(5)

def yolo_to_bbox(image_folder, labels_df: pd.DataFrame):
	image_folder = Path(image_folder)
	converted_bboxes = []

	paths = [i for i in image_folder.glob("*") if i.suffix != ".npy"]
	for image_file in paths:
		image_id = image_file.stem
		if image_id not in labels_df['Image_ID'].values:
			converted_bboxes.append({
				"Image_ID": image_id,
				"class_id": -1,  # Indicating no label
				"xmin": None,
				"ymin": None,
				"xmax": None,
				"ymax": None
			})

	for _, row in labels_df.iterrows():
		all_ids = [i for i in image_folder.glob(f"{row['Image_ID']}*") if i.suffix != ".npy"]
		image_path = image_folder / f"{row['Image_ID']}"
		if all_ids:
			image_path = all_ids[0]

		if image_path.exists():
			img = load_image_(image_path)
			img_width, img_height = img.size

			x_center = row['x_center'] * img_width
			y_center = row['y_center'] * img_height
			width = row['width'] * img_width
			height = row['height'] * img_height

			x_min = x_center - (width / 2)
			y_min = y_center - (height / 2)
			x_max = x_center + (width / 2)
			y_max = y_center + (height / 2)

			converted_bboxes.append({
				"Image_ID": row['Image_ID'],
				"class_id": row['class_id'],
				"xmin": x_min,
				"ymin": y_min,
				"xmax": x_max,
				"ymax": y_max
			})
		else:
			print(f"Image {image_path} not found.")

	return pd.DataFrame(converted_bboxes)

# Example usage
converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax
79,ID_UIprDv,1,380.000256,389.999616,913.999872,1154.999296
30,ID_N1BVjv,2,8.000160,671.999360,285.999840,1086.999680
49,ID_esabf4,1,0.000000,130.999680,1080.000000,750.999600
85,ID_sFXWTT,2,45.999824,1.000064,379.999984,405.000128
4,ID_jTqcfA,0,46.000224,387.999360,309.000096,1014.000000


In [17]:
converted_labels.describe()

,class_id,xmin,ymin,xmax,ymax
count,95.000000,95.000000,95.000000,95.000000,95.000000
mean,1.126316,358.452658,331.631492,1269.147347,1588.589391
std,0.775175,538.727657,481.446497,950.695543,1184.230438
min,0.000000,-0.002016,-0.001632,58.999824,53.000064
25%,1.000000,1.000392,20.500032,498.499968,704.499664
50%,1.000000,114.999840,114.000000,960.000000,1195.000320
75%,2.000000,477.500328,513.499776,1862.000208,2442.499248
max,2.000000,2514.001344,2346.998784,3897.998496,4128.002064


In [18]:
converted_labels["Image_ID"].nunique()

56

In [19]:
converted_labels['class_id'].value_counts()

class_id
1    37
2    35
0    23
Name: count, dtype: int64

In [20]:
class_map

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [21]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

class
cssvd          37
healthy        35
anthracnose    23
Name: count, dtype: int64

In [22]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
54,ID_AECW0W,1,1.000080,318.000640,559.000080,1150.000640,cssvd
25,ID_nauxsB,1,451.000320,565.999360,960.000000,1048.999680,cssvd
32,ID_ET34jY,0,0.000000,722.000000,916.999200,3602.000000,anthracnose
30,ID_N1BVjv,2,8.000160,671.999360,285.999840,1086.999680,healthy
81,ID_yZDVIT,2,87.999808,1.000064,349.999936,375.999936,healthy


In [23]:
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
1660,ID_u0KnR0.jpg,healthy,0.035614,0.755859,1566.140625,469.388672,2473.171875
4127,ID_ET34jY.jpeg,anthracnose,0.000015,1804.687378,1800.000000,2613.281006,1800.000000
6409,ID_dA1GQh.jpg,healthy,0.000004,230.449219,794.179688,326.425781,810.000000
5951,ID_esabf4.jpg,healthy,0.000005,609.609375,0.115356,723.515625,20.764160
2721,ID_Hm1rUc.jpeg,healthy,0.000460,1984.499878,2435.343506,2563.312256,3022.031006


In [24]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
2377,ID_ydI3Pa,anthracnose,0.000034,1125.187500,1923.656250,1443.937500,2309.34375
5238,ID_ka3SdB,healthy,0.000011,0.136475,91.660149,12.085937,170.01561
821,ID_JLxbsU,anthracnose,0.000011,892.500000,255.625000,1242.500000,400.62500


In [25]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels["Image_ID"].unique()}

In [26]:
converted_labels.isna().sum()

Image_ID    0
class_id    0
xmin        0
ymin        0
xmax        0
ymax        0
class       0
dtype: int64

In [27]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

56

In [28]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [29]:
sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
4767,ID_QEGas6,anthracnose,0.000006,255.93750,808.125000,376.562500,911.875000
1763,ID_u0KnR0,anthracnose,0.000243,3986.90625,385.992188,4128.000000,1176.117188
617,ID_jTWHzU,anthracnose,0.000104,0.00000,1649.812378,178.048813,2917.687256


In [30]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
7265,ID_dOxqJU,cssvd,0.000018,14.375,6.5625,1243.750000,227.18750,1
54,ID_MYSxE2,healthy,0.000085,0.000,0.0000,451.828125,390.46875,2
2509,ID_B9K2SI,cssvd,0.000012,419.375,1256.2500,671.875000,1280.00000,1


In [31]:
predictions = get_preds_data(sub, None)

len(predictions)

56

In [32]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

Evaluation metric at: 0.0  score : {'TP': 94, 'FP': 8306, 'FN': 1, 'Precision': 0.01119047619047619, 'Recall': 0.9894736842105263, 'F1 Score': 0.022130665097115948, 'Accuracy': 0.011189144149506012}
Evaluation metric at: 0.06785714285714285  score : {'TP': 85, 'FP': 62, 'FN': 10, 'Precision': 0.5782312925170068, 'Recall': 0.8947368421052632, 'F1 Score': 0.7024793388429751, 'Accuracy': 0.5414012738853503}
Evaluation metric at: 0.1357142857142857  score : {'TP': 81, 'FP': 47, 'FN': 14, 'Precision': 0.6328125, 'Recall': 0.8526315789473684, 'F1 Score': 0.7264573991031391, 'Accuracy': 0.5704225352112676}
Evaluation metric at: 0.20357142857142857  score : {'TP': 78, 'FP': 34, 'FN': 17, 'Precision': 0.6964285714285714, 'Recall': 0.8210526315789474, 'F1 Score': 0.753623188405797, 'Accuracy': 0.6046511627906976}
Evaluation metric at: 0.2714285714285714  score : {'TP': 76, 'FP': 29, 'FN': 19, 'Precision': 0.7238095238095238, 'Recall': 0.8, 'F1 Score': 0.76, 'Accuracy': 0.6129032258064516}
Evalua

In [33]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
80,ID_UIprDv,1,1129.999872,1021.000704,1517.999616,2041.000960,cssvd
20,ID_YQlY8q,1,637.000704,19.999744,1436.000256,663.999488,cssvd
94,ID_Olcex7,2,-0.002016,2224.000800,672.999264,3024.000000,healthy
87,ID_sFXWTT,2,1.000064,182.000000,115.999936,416.000000,healthy
32,ID_ET34jY,0,0.000000,722.000000,916.999200,3602.000000,anthracnose


In [34]:
import torch
from torchmetrics.detection import MeanAveragePrecision

def compute_map(preds, targets):
	"""
	Compute mAP at different IoU thresholds using torchmetrics.
	
	Args:
		preds: List of dicts with 'boxes', 'scores', 'labels' for predictions
		targets: List of dicts with 'boxes', 'labels' for ground truth
	
	Returns:
		Dict containing mAP results for each IoU threshold
	"""
	# Initialize the metric
	metric = MeanAveragePrecision()
	
	# Update metric with predictions and targets
	metric.update(preds, targets)
	
	# Compute the results
	result = metric.compute()
	
	return result

thrs = np.linspace(0.000001, 0.95, 15)
# Example usage
# if __name__ == "__main__":
# Example predictions and targets
# iou_thresholds = [0.5]
for i in thrs:
	preds = list(get_preds_data(sub, i).values())

	targets = list(ground_truth.values())

	# Compute mAP
	results = compute_map(preds, targets)

	# Print results
	print("mAP Results:", i, " - ", results)

/data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/.venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


mAP Results: 1e-06  -  {'map': tensor(0.6144), 'map_50': tensor(0.8328), 'map_75': tensor(0.6795), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.6152), 'mar_1': tensor(0.4811), 'mar_10': tensor(0.7047), 'mar_100': tensor(0.7148), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.7148), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.06785807142857142  -  {'map': tensor(0.6060), 'map_50': tensor(0.8095), 'map_75': tensor(0.6759), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.6060), 'mar_1': tensor(0.4784), 'mar_10': tensor(0.6795), 'mar_100': tensor(0.6795), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.6795), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.13571514285714284  -  {'map': tensor(0.5972), 'map_50': tensor(

In [35]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [36]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
7792,ID_OB1kf9,anthracnose,0.000003,1168.750000,716.250000,1278.750000,877.500000,0
5533,ID_yZNP5q,cssvd,0.000005,767.285156,0.000000,828.457031,0.000000,1
6218,ID_J26p8u,anthracnose,0.000031,3636.187500,3.527344,4123.968750,399.597656,0
4535,ID_sqkRby,healthy,0.000036,1242.500000,770.000000,1280.000000,960.000000,2
3084,ID_RNAyxb,healthy,0.000044,384.375000,740.000000,591.875000,1017.500000,2
